<a href="https://colab.research.google.com/github/rosselladedo/DOG_BREEDS_SEARCH_ENGINE/blob/main/FeatureExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CONNECT TO GDRIVE AND IMPORTS**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
import numpy as np
import time
import itertools
import os
import tensorflow as tf
import tensorflow_hub as hub
import shutil
import PIL
import tqdm

from tqdm import tqdm
from random import random
from ipywidgets import Image
from numpy.linalg import norm
from IPython.display import display
from keras.models import Model
from sklearn.metrics import classification_report

# tensorflow version 2.4.0
from tensorflow import keras

# **USEFULL VARIABLES**

In [ ]:
DOGS_BREEDS = 120
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

In [ ]:
#data directories
BASE_DIR = "/content/gdrive/Shareddrives/MIRCV"
IMAGES_DIR = "/content/gdrive/MyDrive/MIRCV"

DOGS_IMAGES_DIRECTORIES_PATH = os.path.join(IMAGES_DIR, "images/Images")

ALL_IMAGES_DIR = "/content/gdrive/Shareddrives/MIRCV/All_Images"
NOISE_DIR = "/content/gdrive/Shareddrives/Dataset/mirflickr25k/mirflickr"
TEST_PATH = os.path.join(BASE_DIR, "TestSet")

# **IMPORT KAGGLE'S TOKEN**



In [ ]:
###### ESEGUIRE SOLO UNA VOLTA ############

'''
! pip install -q kaggle

from google.colab import files
_ = files.upload()

! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

'''

Saving kaggle.json to kaggle.json


# **EXTRACT IMAGES**

In [ ]:
#upload the distractor on the drive manually
! unzip /content/gdrive/Shareddrives/Dataset/mirflickr25k.zip -d /content/gdrive/Shareddrives/Dataset

Archive:  /content/gdrive/Shareddrives/MIRCV/mirflickr25k.zip


In [ ]:
###### ESEGUIRE SOLO UNA VOLTA ############
'''

! kaggle datasets download -d jessicali9530/stanford-dogs-dataset -p /content/gdrive/Shareddrives/MIRCV

'''

100% 748M/750M [00:06<00:00, 118MB/s]
100% 750M/750M [00:06<00:00, 119MB/s]


In [ ]:
###### ESEGUIRE SOLO UNA VOLTA ############

zip_path = '/content/gdrive/Shareddrives/MIRCV/stanford-dogs-dataset.zip'
!cp "{zip_path}" .
!unzip stanford-dogs-dataset.zip -d '/content/gdrive/Shareddrives/MIRCV/Datasets' 
!rm stanford-dogs-dataset.zip

In [ ]:
################################
#### NON RUNNARE DI NUOVO!! ####
################################

#Per prendere il test set
query_numbers = rand.sample(range(20580), 1000)

def move_images(dest_path = TEST_PATH):
  counter = 1

  for img in tqdm(os.listdir(ALL_IMAGES_DIR)): 
    image_path = os.path.join(ALL_IMAGES_DIR, img)
    
    if counter in query_numbers:
      shutil.move(image_path, dest_path)
    counter = counter + 1

#move_images()

# **EXTRACT FEATURES**

In [ ]:
model = tf.keras.applications.densenet.DenseNet121(
    weights='imagenet', input_tensor=None, input_shape=(224,224,3), 
    pooling='avg', include_top=False
)

model.summary()

In [ ]:
def extract_features():

  features = []
  for img in tqdm(os.listdir(ALL_IMAGES_DIR)):
    image_path = os.path.join(ALL_IMAGES_DIR, img)
    image_pil = tf.keras.utils.load_img(image_path, target_size=IMAGE_SIZE, interpolation='bilinear')
    image_pil = np.array(image_pil)
    image_pil = image_pil.astype(np.float32)
    image_np = tf.keras.applications.densenet.preprocess_input(image_pil)
    image_np = np.expand_dims(image_np, axis=0)  # add batch dimension
    #image_feature_vector = model(image_np).numpy()
    image_feature_vector = model.predict(image_np)
    feat_name = (img, image_feature_vector)
    features.append(feat_name)
  np.savez("/content/gdrive/Shareddrives/Dataset/Features/features_nome_correct.npz", features)
extract_features()

100%|██████████| 19580/19580 [1:09:47<00:00,  4.68it/s]
/usr/local/lib/python3.7/dist-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


In [ ]:
def extract_features_noise():
  
  features = []

  for img in tqdm(os.listdir(NOISE_DIR)):
    image_path = os.path.join(NOISE_DIR, img)
    image_pil = tf.keras.utils.load_img(image_path, target_size=IMAGE_SIZE, interpolation='bilinear')
    image_pil = np.array(image_pil)
    image_pil = image_pil.astype(np.float32)
    image_np = tf.keras.applications.densenet.preprocess_input(image_pil)
    image_np = np.expand_dims(image_np, axis=0)  # add batch dimension
    #image_feature_vector = model(image_np).numpy()
    image_feature_vector = model.predict(image_np)
    feat_name = (img, image_feature_vector)
    features.append(feat_name)
  

  np.savez("/content/gdrive/Shareddrives/Dataset/Features/features_noise_nome_correct.npz", features)

extract_features_noise()

100%|██████████| 25000/25000 [3:22:47<00:00,  2.05it/s]
/usr/local/lib/python3.7/dist-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


In [ ]:
#per inserire le immagini in un'unica cartella, necessario rifare l'estrazione delle immagini dopo però
def move_images(dest_path = ALL_IMAGES_DIR):
  for item in tqdm(os.listdir(DOGS_IMAGES_DIRECTORIES_PATH)): 
    dogs_directories = os.path.join(DOGS_IMAGES_DIRECTORIES_PATH, item)
    for img in tqdm(os.listdir(dogs_directories)):
      image_path = os.path.join(dogs_directories, img)
      shutil.move(image_path, dest_path)

# **EXTRACT FEATURES FINETUNED MODEL**

In [ ]:
model_finetuned = keras.models.load_model('/content/gdrive/Shareddrives/MIRCV/Models/Fine_Tuning_Model6_Flatten_512120_LR0001')

model = keras.models.Model(inputs=model_finetuned.input, outputs=model_finetuned.get_layer('flatten').output)
model.summary()

In [ ]:
def extract_finetuned_features():

  features = []
  for img in tqdm(os.listdir(ALL_IMAGES_DIR)):
    image_path = os.path.join(ALL_IMAGES_DIR, img)
    image_pil = tf.keras.utils.load_img(image_path, target_size=IMAGE_SIZE, interpolation='bilinear')
    image_pil = np.array(image_pil)
    image_pil = image_pil.astype(np.float32)
    image_np = tf.keras.applications.densenet.preprocess_input(image_pil)
    image_np = np.expand_dims(image_np, axis=0)  # add batch dimension
    #image_feature_vector = model(image_np).numpy()
    image_feature_vector = model.predict(image_np)
    feat_name = (img, image_feature_vector)
    features.append(feat_name)
    
  np.savez("/content/gdrive/Shareddrives/Dataset/Features/features_finetuned_nome_correct.npz", features)
  
extract_finetuned_features()

100%|██████████| 19580/19580 [1:09:13<00:00,  4.71it/s]
/usr/local/lib/python3.7/dist-packages/numpy/lib/npyio.py:719: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


In [ ]:
def extract_finetuned_features_noise():
  
  features = []

  for img in tqdm(os.listdir(NOISE_DIR)):
    image_path = os.path.join(NOISE_DIR, img)
    image_pil = tf.keras.utils.load_img(image_path, target_size=IMAGE_SIZE, interpolation='bilinear')
    image_pil = np.array(image_pil)
    image_pil = image_pil.astype(np.float32)
    image_np = tf.keras.applications.densenet.preprocess_input(image_pil)
    image_np = np.expand_dims(image_np, axis=0)  # add batch dimension
    #image_feature_vector = model(image_np).numpy()
    image_feature_vector = model.predict(image_np)
    feat_name = (img, image_feature_vector)
    features.append(feat_name)
  

  np.savez("/content/gdrive/Shareddrives/Dataset/Features/features_finetuned_noise_nome_correct.npz", features)

extract_features_noise()

 42%|████▏     | 10581/25000 [43:32<54:45,  4.39it/s]

# **CONCATENATE FINETUNED FEATURES MODEL PREDICT**

In [ ]:
#concatenate features predict 
path_features_distractors = "/content/gdrive/Shareddrives/Dataset/Features/features_noise_nome_correct.npz"
path_features_images = "/content/gdrive/Shareddrives/Dataset/Features/features_finetuned_nome_correct.npz"

features_im = np.load(path_features_images, allow_pickle=True)
features_images = features_im['arr_0']

features_dis = np.load(path_features_distractors, allow_pickle=True)
features_distractors = features_dis['arr_0']

features_finetuned_concatenate_predict = np.concatenate((features_images, features_distractors))
#print (features_finetuned_concatenate_predict)

np.savez("/content/gdrive/Shareddrives/Dataset/Features/features_finetuned_concatenate.npz",features_finetuned_concatenate_predict)

In [ ]:
#concatenate normative model + distractor

path_features_distractors = "/content/gdrive/Shareddrives/Dataset/Features/features_noise_nome_correct.npz"
path_features_images = "/content/gdrive/Shareddrives/Dataset/Features/features_nome_correct.npz"

features_im = np.load(path_features_images, allow_pickle=True)
features_images = features_im['arr_0']

features_dis = np.load(path_features_distractors, allow_pickle=True)
features_distractors = features_dis['arr_0']

features_finetuned_concatenate_predict = np.concatenate((features_images, features_distractors))
#print (features_finetuned_concatenate_predict)

np.savez("/content/gdrive/Shareddrives/Dataset/Features/features_NM_concatenate.npz",features_finetuned_concatenate_predict)